In [1]:
import $ivy.`org.apache.spark::spark-core:3.2.0`
import $ivy.`org.apache.spark::spark-sql:3.2.0`
import $ivy.`org.apache.spark::spark-hive:3.2.0`

import $ivy.$                                   

import $ivy.$                                  

import $ivy.$                                   

In [16]:
import org.apache.log4j.{LogManager, Level}
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.{functions => F}
import org.apache.spark.sql.expressions.Window

LogManager.getLogger("org").setLevel(Level.ERROR)

val spark = SparkSession.builder.
    appName("app123").
    master("local").
    config("spark.sql.warehouse.dir","/user/hive/warehouse").
    config("spark.hadoop.fs.default.name", "hdfs://localhost:9000").
    enableHiveSupport().
    getOrCreate()

import spark.implicits._

import org.apache.log4j.{LogManager, Level}

import org.apache.spark.sql.SparkSession

import org.apache.spark.sql.{functions => F}

import org.apache.spark.sql.expressions.Window


spark: SparkSession = org.apache.spark.sql.SparkSession@568d52ce
import spark.implicits._

In [32]:
val df = Seq(
    (12345, 1, "123", "ivan", "ivanov", null, "111", "000"),
    (12345, 2, "123", "ivan", "ivanov", null, "111", "000"),
    (12345, 3, "123", "ivan", "ivanov", "ivanovich", "111", ""),
    (12345, 4, "123", "ivan", "ivanov", null, "", "000"),
   
   ).
    toDF("id", "time", "pass", "fname", "lname", "sname", "inn", "snils")

df.show

+-----+----+----+-----+------+---------+---+-----+
|   id|time|pass|fname| lname|    sname|inn|snils|
+-----+----+----+-----+------+---------+---+-----+
|12345|   1| 123| ivan|ivanov|     null|111|  000|
|12345|   2| 123| ivan|ivanov|     null|111|  000|
|12345|   3| 123| ivan|ivanov|ivanovich|111|     |
|12345|   4| 123| ivan|ivanov|     null|   |  000|
+-----+----+----+-----+------+---------+---+-----+



df: org.apache.spark.sql.package.DataFrame = [id: int, time: int ... 6 more fields]

In [34]:
val wf = Window.
    partitionBy("id","pass").
    orderBy("time").
    rowsBetween(Window.unboundedPreceding, Window.currentRow)

val wb = Window.
    partitionBy("id", "pass").
    orderBy("time").
    rowsBetween(Window.currentRow, Window.unboundedFollowing)


df.
    select("id","time","pass","fname","lname", "sname").
    withColumn("fforward", F.last("sname", true).over(wf)).
    withColumn("fback", F.first("fforward", true).over(wb)).show

+-----+----+----+-----+------+---------+---------+---------+
|   id|time|pass|fname| lname|    sname| fforward|    fback|
+-----+----+----+-----+------+---------+---------+---------+
|12345|   1| 123| ivan|ivanov|     null|     null|ivanovich|
|12345|   2| 123| ivan|ivanov|     null|     null|ivanovich|
|12345|   3| 123| ivan|ivanov|ivanovich|ivanovich|ivanovich|
|12345|   4| 123| ivan|ivanov|     null|ivanovich|ivanovich|
+-----+----+----+-----+------+---------+---------+---------+



wf: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@79f765c9
wb: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@4f6fe8a1